In [1]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import random
from scipy.spatial.transform import Rotation as R
from scipy.integrate import simps

In [2]:
class halfplane:
    def __init__(self, strike, dip, plane_point):
        """
        strike: Strike of the half plane in degrees
        dip: Dip of the half plane in  degrees
        plane_point: Point on the plane that will be used to calculate the offset
                     self.d
        In this implementation, the current enters the plate at the edge, and 
        is located at the origin
        """
        
        rad_strike = np.pi * strike / 180
        rad_dip = np.pi * dip / 180
        self.rad_strike = rad_strike
        self.rad_dip = rad_dip
        self.strike = strike
        self.dip = dip
        self.normal = self.fnormal(strike, dip)
        
        # Get plane offset
        # Form: ax + by + cz = d
        self.d = np.dot(self.normal, plane_point)
        
        # Init unit axis vectors
        self.x_h, self.y_h, self.z_h = np.array([1,0,0]), \
                                       np.array([0,1,0]), \
                                       np.array([0,0,1])

        # Defines the rotation to the frame of reference used for all the math
        # Note that this frame of reference is different from the one in Pai and
        # Yeoh 1993
        # 1.) Rotate y axis about z to the plunge vector projected onto x-y
        # 2.) Rotate y' axis about x' axis to the plunge vector
        # 3.) Rotate about y'' such that the Z vector lies on the plane
        
        #self.rotation = R.from_euler()
        
    def in_space(self, pt):
        """
        Returns bool value if pt is within the half plane defined by the class
        """
        # This function below evaluates z coordinate of self.div_line at point [x, y]
        z_func_line = lambda x, y: -np.dot(np.array([x, y]), self.div_line[:2]) / self.div_line[2]
        if np.dot(self.normal, pt) < 1e-6 and pt[2] < z_func_line(pt[0], pt[1]):
            return True
        else:
            return False
    
    def fnormal(self, strike, dip):
        """
        Returns the normal vector given strike and dip
        """
        deg_to_rad = lambda x: x * np.pi / 180
        r_strike = deg_to_rad(strike)
        r_dip = deg_to_rad(dip)
        n = np.array([
            np.sin(r_dip) * np.cos(r_strike),
            -np.sin(r_dip) * np.sin(r_strike),
            -np.cos(r_dip)
        ])
        return n
    
    

In [3]:
halfplane(45, dip=50, plane_point=np.array([5,5,5]))

TypeError: from_euler() missing 2 required positional arguments: 'seq' and 'angles'

In [6]:
x_h, y_h, z_h = np.array([1,0,0]), \
                np.array([0,1,0]), \
                np.array([0,0,1])

In [7]:
BH_DIP = 54
BH_AZIMUTH = 18

In [8]:
# Convenience function init
def get_cones(pt, vec):
    """
    Return the cone with origin at pt, vector
    """
    cones = go.Cone(x=[pt[0]], y=[pt[1]], z=[pt[2]], 
                    u=[vec[0]], v=[vec[1]], w=[vec[2]], 
                    sizemode='scaled',
                    sizeref=0.1,
                    anchor='tail',
                   )
    return cones

def basis(pt, *args):
    return [get_cones(pt, v) for v in args]

def line_output(start, length, deg_azimuth, deg_dip):
    """
    Calculate the endpoint of a line from start and endpoint specified by 
    length, deg_azimuth, and deg_dip
    """
    start = np.array(start)
    azi = deg_azimuth * np.pi / 180
    phi = (deg_dip + 90) * np.pi / 180
    # Translate out of spherical coordinates
    x,y,z = np.sin(azi), np.cos(azi), np.cos(phi)
    dir_hat = np.array([x, y, z]) / np.linalg.norm(np.array([x, y, z]))
    direction = length * dir_hat
    endpoint = start + direction
    return endpoint

# Try a line from [0,0,0] with length 1 and turn it into an object for plotting
bh_start = [0,0,0]
bh_end = line_output(bh_start, 1, BH_AZIMUTH, BH_DIP)
bh = [go.Scatter3d(x=[bh_start[0], bh_end[0]],
                  y=[bh_start[1], bh_end[1]],
                  z=[bh_start[2], bh_end[2]],
                  name='BH',
                  mode='lines+markers')]


NameError: name 'line_output' is not defined